In [ ]:
import os, sys

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

to_remove = []
d_to_remove = workbookDir
for i, p in enumerate(sys.path):
    try:
        if p == "":
            to_remove.append(i)
        elif os.path.samefile(p, d_to_remove):
            to_remove.append(i)
    except:
        pass

for i in reversed(to_remove):
    try:
        sys.path.pop(i)
    except:
        pass

sys.path.insert(0, "/home/guangyunhan/onnxruntime/build_rocm/Release/build/lib")



In [ ]:
import onnxruntime as ort
import onnx
import numpy as np

In [ ]:
input = onnx.helper.make_tensor_value_info("input", onnx.TensorProto.FLOAT16, ["batchsize", 512, 768])
attn_mask = onnx.helper.make_tensor_value_info("attn_mask", onnx.TensorProto.INT32, ["batchsize", 512])
output = onnx.helper.make_tensor_value_info("output", onnx.TensorProto.FLOAT16, ["batchsize", 512, 768])

np.random.seed(1)
# qkv_weight = onnx.helper.make_tensor("qkv_weight", onnx.TensorProto.FLOAT16, [768, 2304], np.random.randn(*[768, 2304]))
qkv_weight = onnx.helper.make_tensor("qkv_weight", onnx.TensorProto.FLOAT16, [768, 2304], np.random.normal(-7.5e-05, 0.09015, [768, 2304]))
# qkv_weight = onnx.helper.make_tensor("qkv_weight", onnx.TensorProto.FLOAT16, [768, 2304], np.zeros([768, 2304]))
# qkv_bias = onnx.helper.make_tensor("qkv_bias", onnx.TensorProto.FLOAT16, [2304], np.random.random([2304]))
qkv_bias = onnx.helper.make_tensor("qkv_bias", onnx.TensorProto.FLOAT16, [2304], np.zeros([2304]))



node = onnx.helper.make_node("Attention", inputs=["input", "qkv_weight", "qkv_bias", "attn_mask"], outputs=["output"], domain="com.microsoft", num_heads=12)

graph = onnx.helper.make_graph([node], "Attn", [input, attn_mask], [output], initializer=[qkv_weight, qkv_bias])

model = onnx.helper.make_model(graph, producer_name="tmp", opset_imports=[
    onnx.helper.make_opsetid('com.microsoft', 1), 
    onnx.helper.make_opsetid('ai.onnx.ml', 1), 
    onnx.helper.make_opsetid('', 14),
])

print(onnx.checker.check_model(model))

In [ ]:
sess = ort.InferenceSession(
    model.SerializeToString(), providers=[
    ("ROCMExecutionProvider", {"tunable_op_enabled": "0"})
])


In [ ]:
input = np.random.randn(64, 512, 768)
input = input.astype(np.float16)

In [ ]:
os.environ["ORT_ATTENTION_USE_GEMM_RCR_BIAS_PERMUTE"] = "1"
os.environ["ORT_ATTENTION_USE_BATCHED_GEMM_SOFTMAX_GEMM_PERMUTE"] = "1"

o11 = sess.run(
    output_names = [node.name for node in sess.get_outputs()],
    input_feed = { "input": input, "attn_mask": np.ones([64, 512], dtype=np.int32)}
)[0]


In [ ]:
os.environ["ORT_ATTENTION_USE_GEMM_RCR_BIAS_PERMUTE"] = "0"
os.environ["ORT_ATTENTION_USE_BATCHED_GEMM_SOFTMAX_GEMM_PERMUTE"] = "1"

o01 = sess.run(
    output_names = [node.name for node in sess.get_outputs()],
    input_feed = { "input": input, "attn_mask": np.ones([64, 512], dtype=np.int32)}
)[0]

In [ ]:
os.environ["ORT_ATTENTION_USE_GEMM_RCR_BIAS_PERMUTE"] = "1"
os.environ["ORT_ATTENTION_USE_BATCHED_GEMM_SOFTMAX_GEMM_PERMUTE"] = "0"

o10 = sess.run(
    output_names = [node.name for node in sess.get_outputs()],
    input_feed = { "input": input, "attn_mask": np.ones([64, 512], dtype=np.int32)}
)[0]

In [ ]:
os.environ["ORT_ATTENTION_USE_GEMM_RCR_BIAS_PERMUTE"] = "0"
os.environ["ORT_ATTENTION_USE_BATCHED_GEMM_SOFTMAX_GEMM_PERMUTE"] = "0"


o00 = sess.run(
    output_names = [node.name for node in sess.get_outputs()],
    input_feed = { "input": input, "attn_mask": np.ones([64, 512], dtype=np.int32)}
)[0]

In [ ]:
# %timeit _ = sess.run(output_names = [node.name for node in sess.get_outputs()], input_feed = { "input": input, "attn_mask": np.ones([64, 512], dtype=np.int32)})

In [ ]:
ref = o00
my = o01
diff = ref - my

In [ ]:
print(ref)
print(my)

In [ ]:
diff

In [ ]:
denorm = ref.copy()
denorm[denorm == 0] = float("inf")
rtol = np.abs(diff / denorm)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.show(rtol[0])

In [ ]:
rtol_no_inf = 

In [ ]:
rtol[2].max()

In [ ]:
plt.imshow(np.abs(diff[0]) > 1)

In [ ]:
os.environ["KERNEL_EXPLORER_BUILD_DIR"] = "/home/guangyunhan/onnxruntime/build_rocm/Release"
sys.path.insert(0, "/home/guangyunhan/onnxruntime/onnxruntime/python/tools/kernel_explorer/kernels")

In [ ]:
import kernel_explorer as ke
import numpy as np
import torch

import matplotlib.pyplot as plt

In [ ]:
def multinormal_distribution(num_distribution, num_element_per_dist):
    arrays = []
    for i in range(num_distribution):
        mean = np.random.randn()
        std = np.random.rand()
        arrays.append(np.random.normal(mean, std, (num_element_per_dist,)))
    return np.array(arrays)

In [ ]:
batchsize = 64
seq_len = 512
num_heads = 12
head_dim = 64
scale = 1.0

np.random.seed(1)
rand = multinormal_distribution(batchsize * num_heads * seq_len, 3 * head_dim)

q = rand[:, 0*head_dim:1*head_dim].reshape(batchsize, num_heads, seq_len, head_dim)
k = rand[:, 1*head_dim:2*head_dim].reshape(batchsize, num_heads, seq_len, head_dim)
# v = rand[:, 2*head_dim:3*head_dim].reshape(batchsize, num_heads, seq_len, head_dim)
v = np.random.uniform(-0.1, 0.1, (batchsize, num_heads, seq_len, head_dim))

attn_before_softmax = torch.matmul(torch.Tensor(q), torch.Tensor(k).transpose(2, 3)) * scale
attn = torch.softmax(attn_before_softmax, dim=-1)
ref = torch.permute(torch.matmul(attn, torch.Tensor(v)), [0, 2, 1, 3]).numpy()

q = q.astype(np.float16)
k = k.astype(np.float16)
v = v.astype(np.float16)
my = np.zeros((batchsize, seq_len, num_heads, head_dim), dtype=np.float16)

dev_q = ke.DeviceArray(q)
dev_k = ke.DeviceArray(k)
dev_v = ke.DeviceArray(v)
dev_my = ke.DeviceArray(my)

op = ke.BatchedGemmSoftmaxGemmPermute_half(dev_q, dev_k, dev_v, dev_my, batchsize, seq_len, num_heads, head_dim, scale)
op.Run()
dev_my.UpdateHostNumpyArray()

diff = ref - my


In [ ]:
plt.imshow((np.abs(rtol) > 0.005)[0])

In [ ]:
diff = diff.reshape(64, 512, 768)

In [ ]:
np.sum(np.abs(rtol) > 0.01)

In [ ]:
np.abs(diff).argmax()

In [ ]:
rtol = (ref - my) / ref
rtol = rtol.reshape(64, 512, 768)

In [ ]:
rtol.reshape(-1)[6217159]

In [ ]:
ref.reshape(-1)[6217159]

In [ ]:
my.reshape(-1)[6217159]

In [ ]:
import os
import sys

os.environ["KERNEL_EXPLORER_BUILD_DIR"] = "/home/guangyunhan/onnxruntime-ke/build_rocm/Release"
sys.path.insert(0, "/home/guangyunhan/onnxruntime-ke/onnxruntime/python/tools/kernel_explorer/kernels")

import kernel_explorer as ke
import numpy as np

In [ ]:
batch = 64
M = 512
K = 768
N = 3072
O = 768

np.random.seed(0)
a = 0.01 * np.random.uniform(size=(batch, M, K))
b = 0.01 * np.random.uniform(size=(1, K, N))
# bias = 0.1 * np.ones((N,))
bias = 0.1 * np.zeros((M, N,))
# bias = np.random.uniform(size=(N,))
c = 0.1 * np.random.uniform(size=(1, N, O))

ref = (a @ b + bias) @ c

In [ ]:
a = a.astype(np.float16)
b = b.astype(np.float16)
bias = bias.astype(np.float16)
c = c.astype(np.float16)
my = np.zeros_like(ref, dtype=np.float16)

dev_a = ke.DeviceArray(a)
dev_b = ke.DeviceArray(b)
dev_bias = ke.DeviceArray(bias)
dev_c = ke.DeviceArray(c)
dev_my = ke.DeviceArray(my)

op = ke.BatchedGemmBiasGeluGemm_half(dev_a, dev_b, dev_bias, dev_c, dev_my, M, K, N, O, batch)
op.Run()
dev_my.UpdateHostNumpyArray()

rdiff = np.abs((ref - my) / ref)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(rdiff[-1] > 0.005)